<a href="https://colab.research.google.com/github/burakericok/hard_disk_crits/blob/main/visualize_critical_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Critical Points of Configuration Spaces of Hard Disks**

This is a database that contains the critical configurations of hard disks on the hexagonal torus, where the number of hard disks is $n = 2 \ldots 12$. A critical configuration is one where disk contacts prevent any collective motion that would allow the radius of all disks to increase linearly with the extent of the motion. Equivalently, a critical configuration is one that has a mechanically-balanced contact graph. The torus is the regular hexagon of edge length $1 / \sqrt{3}$ with opposite edges identified.

## **Security Statement**
Connecting to a Jupyter notebook server running on your local machine can provide many benefits. With these benefits come serious potential risks. By connecting to a local runtime, you are allowing the Colaboratory frontend to execute code in the notebook using the local resources on your machine. This means that the notebook could:

- Invoke arbitrary commands (e.g. "rm -rf /")
- Access the local file system
- Run malicious content on your machine

Before attempting to connect to a local runtime, make sure you trust the authors of the notebook and ensure you understand what code is being executed. For more information on the Jupyter notebook server's security model, consult [Jupyter's documentation](https://jupyter-notebook.readthedocs.io/en/stable/security.html).

## **Usage**
The capabilities of this notebook are contained in two functions, `import_database` and `plot_crit`. These are defined in the `Define necessary functions` cell (double clicking expands the cell if you would like to examine the code). Run this cell first.

The next cell imports the database of critical points for the desired number of disks, passed as the single argument to `import_database`. Critical point databases are available for $n=2 \ldots 12$. Running this cell displays the (truncated) table, which you can search for particular values of the index or radius by pressing the `Filter` button. The truncated columns contain information about the positions of the disks and the contacts between the disks.

The next cell displays the adjacency matrix and disk configuration of a selected critical point, passed as the second argument to `plot_crit`. The value of this argument must be an integer that appears in the `crit_number` column of the preceeding data table.

For example, to plot the second critical point for seven disks:
- Run the `Define necessary functions` cell (only needs to be done once)
- Change the argument of `import_database` in the next cell to `7` and run the cell 
- Change the second argument of `plot_crit` in the next cell to `2` and run the cell

## **References**
Research into the topology of the configuration space of hard disks is ongoing, but the references below provide some context for this project.

- G. Carlsson, J. Gorham, M. Kahle, J. Mason, [Computational topology for configuration spaces of hard disks](https://doi.org/10.1103/PhysRevE.85.011303), Phys. Rev. E 85 (2012): 011303.
- Y. Baryshnikov, P. Bubenik, M. Kahle, [Min-type Morse theory for configuration spaces of hard spheres](https://doi.org/10.1093/imrn/rnt012), International Mathematics Research Notices 2014.9 (2014): 2577–2592.
- H. Alpert, M. Kahle, R. MacPherson, [Configuration spaces of disks in an infinite strip](https://arxiv.org/abs/1908.04241), arXiv:1908.04241.

## **Contributors**
The contributors to this project are (by alphabetical order of last name):

- Ozan Ericok (oericok@ucdavis.edu)
- Matthew Kahle (kahle.70@osu.edu)
- Jeremy Mason (jkmason@ucdavis.edu)
- Katherine Ritchey (ritcheka@mountunion.edu)

Please contact Matthew Kahle (kahle.70@osu.edu) or Jeremy Mason (jkmason@ucdavis.edu) for more information.

## **License**
The functions included in this archive are licenced under the [GNU General
Public License, Version 3](https://www.gnu.org/licenses/gpl-3.0.en.html).

## **Acknowledgements**
This material is based upon work supported by the National Science Foundation under Grant No. 1839370. Any opinions, findings, and conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the National Science Foundation.

In [1]:
#@title Define necessary functions
#Defines the import_database function
import pandas as pd
from google.colab import data_table

def import_database(n_disks):
  filename = 'https://raw.githubusercontent.com/burakericok/hard_disk_crits/main/' + str(n_disks) + 'disk.csv'
  crit_database = pd.read_csv(filename)
  return crit_database

#Defines the plot_crit function
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib

def plot_crit(database, id):
  row = database.loc[database['crit_number'] == id];
  coords = row[row.columns[row.columns.to_series().str.contains('coords')]]
  radius = row.iloc[0]['radius']
  contact_graph = row[row.columns[row.columns.to_series().str.contains('contact_graph')]]

  # constants
  cmap = plt.get_cmap("Pastel1")
  colorscale = [];
  for a in range(9):
    colorscale.append('rgb' + str(cmap(a)[0:3]))
  cmap = plt.get_cmap("Pastel2")
  for a in range(8):
    colorscale.append('rgb' + str(cmap(a)[0:3]))
    
  n_disk = np.int(np.size(coords)/2)
  v1 = np.array([1., 0.])
  v2 = np.array([0.5, np.sqrt(3.) / 2.])
  v3 = np.array([-0.5, np.sqrt(3.) / 2.])
  shift = [v1, v2, v2 - v1, -v1, -v2, v1 - v2, 2. * v1, v1 + v2, 2. * v2, 2. * v2 - v1, 2. * (v2 - v1), v2 - 2. * v1, -2. * v1, -(v1 + v2), -2. * v2, v1 - 2. * v2, 2. * (v1 - v2), 2. * v1 - v2]

  # Compute the adjacency matrix
  adjacency_matrix = np.zeros((n_disk, n_disk)) 
  adjacency_matrix[np.triu_indices(n_disk, 1)] = contact_graph
  adjacency_matrix = adjacency_matrix + adjacency_matrix.transpose()
  print('adjacency_matrix:\n', adjacency_matrix)

  # Generate the images
  ximages = np.zeros((np.size(shift, 0) + 1, 2 * n_disk))
  ximages[0] = coords
  for a in range(np.size(shift, 0)):
    ximages[a + 1] = coords + np.matlib.repmat(shift[a], 1, n_disk)
 
  # plotting
  fig = go.Figure()

  # Add colored disks
  shapes = [];
  for a in range(n_disk):
    for b in range(np.int(np.size(ximages,0))):
      shapes.append(
        dict(
          type = "circle",
          xref = "x",
          yref = "y",
          #fillcolor = colorscale[a],
          x0 = ximages[b, 2 * a] - radius,
          y0 = ximages[b, 2 * a + 1] - radius,
          x1 = ximages[b, 2 * a] + radius,
          y1 = ximages[b, 2 * a + 1] + radius,
          line_color = colorscale[a],
          line_width = 5
        )
      )
  fig.update_layout(shapes = shapes)

  # Add disk centers
  x = ximages[:, 0::2]
  y = ximages[:, 1::2]
  x = np.reshape(x, np.size(x))
  y = np.reshape(y, np.size(y))

  fig.add_trace(
    go.Scatter(
      x = x,
      y = y,
      mode = 'markers',
      marker = dict(
        color='black',
        size = 10,
        symbol = 'square',
      ),
      showlegend = False
    )
  )

  # Add edges
  # find the edge list
  ximages = np.reshape(ximages, (np.int(np.size(ximages, 0) * n_disk), 2))
  d = np.zeros((np.int(np.size(ximages, 0) * (np.size(ximages, 0) - 1) / 2), 3))
  c = 0
  a = 0
  while (a < np.size(ximages, 0)):
    b = a + 1;
    p1 = ximages[a];
    while(b < np.size(ximages, 0)):
        p2 = ximages[b]
        d[c] = [a, b, np.linalg.norm(p1 - p2)]
        c = c + 1
        b = b + 1
    a = a + 1

  edge_list = d[d[:, 2] < 2.001 * radius, :]
  pairs = edge_list[:, [0, 1]]

  x_lines = list()
  y_lines = list()
  for p in pairs:
    for a in range(2):
      x_lines.append(ximages[np.int(p[a]), 0])
      y_lines.append(ximages[np.int(p[a]), 1])
    x_lines.append(None)
    y_lines.append(None)

  fig.add_trace(
    go.Scatter(
      x = x_lines,
      y = y_lines,
      mode = 'lines',
      line = dict(
        color = 'black',
        width = 3
      ),
      showlegend = False
    )
  )

  # Random plots for legend
  for a in range(n_disk):
    fig.add_trace(
      go.Scatter(
        x = [100], 
        y = [100],
        mode = 'markers',
        marker = dict(
          size=[10],
          color=colorscale[a]
        ),
        name = 'Disk ' + str(a + 1)
      )
    )

  # Final touches
  fig.update_layout(
    width = 1000, 
    height = 1000,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    ),    
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    plot_bgcolor = 'rgba(0, 0, 0, 0)'
  )
  fig.update_xaxes(range = [-3., 3.])
  fig.update_yaxes(range = [-3., 3.])

  fig.show()

In [ ]:
crit_database = import_database(6)
data_table.DataTable(crit_database, include_index=False, num_rows_per_page=10, max_columns=7, min_width='500px')

In [ ]:
plot_crit(crit_database, 6)